In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import re
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
#Getting Data
data_df = pd.read_csv('/content/drive/MyDrive/FinalBalancedDataset.csv')
data_df = data_df.sample(frac=1, random_state=42) # shuffle with random_state=42 for reproducibility
data_df.head()

data_df = data_df.drop(columns=['Unnamed: 0'])
data_df.tail(15)

,Toxicity,tweet
16023,0,@user are we all not tired of these #accraflo...
44131,1,Just straight like that. No chaser. And the bi...
47191,1,RT @FreddieGibbs: It's cold as penguin pussy.
21962,0,@user @user @user @user @user this is a msu t...
37194,1,@Yummys_World &#8220;@ThoughtsOfRandy @STROCOR...
16850,0,"the most cheesiest picture ever, but i love it..."
6265,0,jst the past few weeks a close family frnd was...
54886,0,Wicked Oreo
11284,0,from #bracelet #vintagestyle w/free #butterfl...
44732,1,"Marijuana, hydro, pussy hoe, ass titties"


In [ ]:
data_df['None'] = (data_df.drop(columns=['id', 'comment_text']).sum(axis=1) == 0).astype(int)

KeyError: "['id', 'comment_text'] not found in axis"

In [ ]:
data_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,None
119105,7ca72b5b9c688e9e,"Geez, are you forgetful! We've already discus...",0,0,0,0,0,0,1
131631,c03f72fd8f8bf54f,Carioca RFA \n\nThanks for your support on my ...,0,0,0,0,0,0,1
125326,9e5b8e8fc1ff2e84,"""\n\n Birthday \n\nNo worries, It's what I do ...",0,0,0,0,0,0,1
111256,5332799e706665a6,Pseudoscience category? \n\nI'm assuming that ...,0,0,0,0,0,0,1
83590,dfa7d8f0b4366680,"(and if such phrase exists, it would be provid...",0,0,0,0,0,0,1


In [ ]:
#removing unnecesary symbols
X = [re.sub('http://\S+|https://\S+', '', text) for text in data_df['comment_text']]
X = [text.replace('@user', '') for text in X]
y = data_df.drop(columns=['comment_text', 'id'])
y = y['None']
y.tail(15)

64820     0
67221     1
41090     1
16023     1
126324    1
112727    1
87498     1
137337    1
54886     1
110268    1
119879    1
103694    1
131932    0
146867    1
121958    1
Name: None, dtype: int64

In [ ]:
none_rows = y[y == 1]

In [ ]:
none_rows.tail(15)

59735     1
769       1
67221     1
41090     1
16023     1
126324    1
112727    1
87498     1
137337    1
54886     1
110268    1
119879    1
103694    1
146867    1
121958    1
Name: None, dtype: int64

In [ ]:
to_remove = none_rows.sample(n=120000, random_state=1).index

In [ ]:
data_df = data_df.drop(index=to_remove)

In [ ]:
data_df['None'].sum()

23346

In [ ]:
toxic_rows_series.tail(15)

NameError: name 'toxic_rows_series' is not defined

In [ ]:
non_zero_rows.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate,None


In [ ]:
y.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate,None
119105,0,0,0,0,0,0,1
131631,0,0,0,0,0,0,1
125326,0,0,0,0,0,0,1
111256,0,0,0,0,0,0,1
83590,0,0,0,0,0,0,1


In [ ]:
activity_counts = [y[column].sum() for column in y.columns]
activity_counts

AttributeError: 'Series' object has no attribute 'columns'

In [ ]:
y.head()

119105    1
131631    1
125326    1
111256    1
83590     1
Name: None, dtype: int64

In [ ]:
#train,val and labels
train_sentences, val_sentences, train_labels, val_labels = train_test_split(np.array(X),np.array(y),test_size=0.1,random_state=42)

In [ ]:
train_sentences[:10], train_labels[:10]

(array(['"DanilovicAlright, we\'ll stay away from value-judgement, although when you\'ve got a documented rap sheet as long as Danilovic\'s, ""surly"" is almost a compliment. Yes, Blic is a reliable source. It\'s one of the high-circulation dailies in Serbia. Yes, it\'s a middle-market tabloid, but it doesn\'t invent stories. It\'s very similar to Britain\'s Daily Mail. Plus the case of Danilovic beating up referee Juras is well-documented in Serbian media as well as the resulting sanctions, both basketball-related and civic.  "',
        '" Jeepers  I just came looking for info on Madame Sosostris and I found this preening custerfluck instead. I thought I remembered the character from a modern opera called ""A Midsummer Marriage"", and then discovered the name was most likely inspired by T.S. Eliot\'s poem, ""The Wasteland"", which, in turn, was inspred by Huxley?  Edit away. I just thought that if somebody came to this page it might be nice if they actually found some info on Madame 

In [ ]:
#Convert text to vectors for better performance
from tensorflow.keras.layers import TextVectorization
text_vectorizer = TextVectorization(max_tokens=200000, # how many words in the vocabulary (all of the different words in your text)
                                    standardize="lower_and_strip_punctuation", # how to process text
                                    split="whitespace", # how to split tokens
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None) # how long should the output sequence of tokens be?
                                    # pad_to_max_tokens=True) # Not valid if using max_tokens=None
max_vocab_length = 10000
max_length = 15 #How many words our network sees

In [ ]:
inputs = tf.keras.Input(shape=(1,), dtype=tf.string)
outputs = text_vectorizer(inputs)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.save("/content/text_vectorizer.h5")

NotImplementedError: Save or restore weights that is not an instance of `tf.Variable` is not supported in h5, use `save_format='tf'` instead. Received a model or layer TextVectorization with weights [<keras.src.layers.preprocessing.index_lookup.VocabWeightHandler object at 0x7a12386e95a0>]

In [ ]:
text_vectorizer.save('/content/text_vectorizer')

AttributeError: 'TextVectorization' object has no attribute 'save'

In [ ]:
words_in_vocab = text_vectorizer.get_vocabulary()

words_in_vocab[:10]

['', '[UNK]', 'the', 'to', 'of', 'and', 'a', 'you', 'i', 'is']

In [ ]:
tf.random.set_seed(42)

embedding = layers.Embedding(input_dim=len(words_in_vocab), # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length, # how long is each input
                             name="embedding_1")

embedding

In [ ]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

# Ensure the text_vectorizer is adapted to the training data
text_vectorizer.adapt(train_sentences)

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
I had to take a time-out, but I'm back and watching. And say, if you have a chance to see it, it's a pretty clever and funny movie!      

Embedded version:


<tf.Tensor: shape=(1, 28, 128), dtype=float32, numpy=
array([[[ 0.0034801 , -0.04895495, -0.03692429, ..., -0.01571351,
          0.04104565,  0.01658935],
        [ 0.01146383, -0.04152433, -0.0185867 , ..., -0.02021475,
         -0.0102347 , -0.01469932],
        [ 0.02204031,  0.01558527, -0.00228897, ..., -0.04431289,
         -0.04630125,  0.02842445],
        ...,
        [ 0.00187045, -0.00292064, -0.02920045, ...,  0.03367729,
         -0.00213055, -0.02419078],
        [-0.035806  ,  0.03460192,  0.015265  , ..., -0.02754443,
         -0.02669474,  0.03884765],
        [-0.04804295,  0.00872707, -0.04952095, ..., -0.01644617,
         -0.03949771, -0.02464316]]], dtype=float32)>

In [ ]:
def build_model():
  inputs = layers.Input(shape=(1,), dtype = 'string')
  x = text_vectorizer(inputs)
  x = embedding(x)
  x = layers.GlobalAveragePooling1D()(x)
  outputs = layers.Dense(1, activation='sigmoid')(x)

  model = tf.keras.Model(inputs, outputs, name='model_dense')
  return model

In [ ]:
text_vectorizer.save('/content/text_vectorizer.h5')

AttributeError: 'TextVectorization' object has no attribute 'save'

In [ ]:
model = build_model()

In [ ]:
model.compile(loss='binary_crossentropy',
               optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001),
               metrics = ["accuracy"])

In [ ]:
model.summary()

Model: "model_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (Text  (None, None)              0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, None, 128)         25600000  
                                                                 
 global_average_pooling1d_2  (None, 128)               0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 25600129 (97.66 MB)
Trainable params: 256

In [ ]:
hist = model.fit(train_sentences, train_labels,
           epochs = 1,
           validation_data = (val_sentences, val_labels))

4488/4488 [==============================] - 131s 29ms/step - loss: 0.2691 - accuracy: 0.9044 - val_loss: 0.2213 - val_accuracy: 0.9158


In [ ]:
test = pd.read_csv("/content/drive/MyDrive/toxic_comment/test.csv").drop(columns=['id'])

In [ ]:
test.head()

,comment_text
0,Yo bitch Ja Rule is more succesful then you'll...
1,== From RfC == \n\n The title is fine as it is...
2,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,":If you have a look back at the source, the in..."
4,I don't anonymously edit articles at all.


In [ ]:
model.predict(["hello"])

1/1 [==============================] - 0s 30ms/step


array([[0.44177848]], dtype=float32)

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save('/content/drive/MyDrive/model_languagev5', save_format='tf')